<h2> Multi-variate Gaussian Distribution </h2>

The multivariate normal distribution of a k-dimensional random vector $ \large X =(X_{1},\ldots ,X_{k})^{T}$ is given by:<br>
$ \large  X \sim  N \boldsymbol (\mu, \boldsymbol \Sigma) $
with k-dimensional mean vector <br>
$ \large \boldsymbol \mu = E[X] = [E[X_{1}], E[X_{2}],\ldots , E[X_{k}]]^{T} $

and $ \large k \times k $ covariance matrix $\large \Sigma $

<h2> Gaussian Process </h2>
* GP is Gaussian distribution over functions. We are learning a *distribution* from empirical data
* Start with prior $\large f_{prior} \sim  N  (\boldsymbol 0, \boldsymbol K) $
* Compute the posterior from training samples by doing a bayesian update<br>
$ \large  f_{posterior} \sim  N \boldsymbol (\mu_{*}, \Sigma_{*}) $ where <br>
$ \large \mu_* = K_* K^{-1} y $ <br>
$ \large \Sigma_* = K_{∗∗} − K_∗ K^{−1} K^T $


In [ ]:
import numpy as np
from ipywidgets import *
import bqplot.pyplot as plt
from bqplot import *

In [ ]:
def squared_exponential(x1, x2, sigma=1., l=1.):
    z = (x1 - x2[:, np.newaxis]) / l
    return sigma**2 * np.exp(-.5 * z ** 2)

In [ ]:
def gp_regression(X_train, y_train, X_test,
                  kernel=squared_exponential,
                  sigma_noise=.1,
                  params=dict(sigma=1., l=1.)):
    # compute the kernel matrices for train, train_test, test combinations
    K = kernel(X_train, X_train, **params)
    K_s = kernel(X_train, X_test, **params)
    K_ss = kernel(X_test, X_test, **params)
    
    n, p = len(X_train), len(X_test)
    
    # compute the posterior mean and cov
    mu_s = np.dot(K_s, np.linalg.solve(K + sigma_noise**2 * np.eye(n), y_train))
    cov_s = K_ss - np.dot(K_s, np.linalg.solve(K + sigma_noise**2 * np.eye(n), K_s.T))
    
    # prior and posterior moments
    mu_prior, cov_prior = np.zeros(p), K_ss
    mu_post, cov_post = mu_s, cov_s + sigma_noise**2
    
    return dict(prior=(mu_prior, cov_prior), 
                posterior=(mu_post, cov_post))

In [ ]:
kernel = squared_exponential
params = dict(sigma=1., l=1.)

X_test = np.arange(-5, 5, .05)
p = len(X_test)
K_ss = kernel(X_test, X_test, **params)
mu_prior, cov_prior = np.zeros(p), K_ss

N = 5
f_priors = np.random.multivariate_normal(mu_prior, cov_prior, N)

In [ ]:
fig_margin=dict(top=60, bottom=40, left=50, right=0)

fig = plt.figure(title='Gaussian Process Regression', 
                 layout=Layout(width='1000px', height='600px'),
                 fig_margin=fig_margin)

plt.scales(scales={'x': LinearScale(min=-5, max=5),
                   'y': LinearScale(min=-5, max=5)})

train_scat = plt.scatter([], [], colors=['salmon'], 
                         enable_move=True,
                         interactions={'click': 'add'},
                         marker_size=1, marker='square')

prior_lines = plt.plot(X_test, f_priors, stroke_width=1, colors=['#ccc'])
posterior_lines = plt.plot(X_test, [], stroke_width=1)

mean_line = plt.plot(X_test, [], 'm')
std_bands = plt.plot(X_test, [],
                     fill='between',
                     fill_colors=['orange'],
                     fill_opacities=[.2], stroke_width=0)
plt.xlabel('X')
plt.ylabel('Y')

# reset btn
reset_button = Button(description='Reset Points', button_style='success')
reset_button.layout.margin = '20px 0px 0px 70px'

# controls for the plot
f_priors_cb = Checkbox(description='Display 5 Priors?')
f_posteriors_cb = Checkbox(description='Display 5 Posteriors?')
std_bands_cb = Checkbox(description='Display Std Bands?')
check_boxes = [f_priors_cb, f_posteriors_cb, std_bands_cb]

# link widgets
_ = jslink((f_priors_cb, 'value'), (prior_lines, 'visible'))
_ = jslink((f_posteriors_cb, 'value'), (posterior_lines, 'visible'))
_ = jslink((std_bands_cb, 'value'), (std_bands, 'visible'))

def update_reg_line(change):
    global mu_post, sig_post
    
    fig.animation_duration = 0
    X_train = train_scat.x
    y_train = train_scat.y
    
    fig.animation_duration = 1000

    gp_res = gp_regression(X_train, y_train, X_test, sigma_noise=0.04)
    mu_post, cov_post = gp_res['posterior']
    
    # simulate N samples from the posterior distribution
    posterior_lines.y = np.random.multivariate_normal(mu_post, cov_post, N)
    sig_post = np.sqrt(np.diag(cov_post))

    # update the regression line to the mean of the posterior distribution
    mean_line.y = mu_post
    
    # update the std bands to +/- 2 sigmas from the posterior mean
    std_bands.y = [mu_post - 2 * sig_post, mu_post + 2 * sig_post]

train_scat.observe(update_reg_line, names=['x', 'y'])

def reset_points(*args):
    with train_scat.hold_trait_notifications():
        train_scat.x = []
        train_scat.y = []
reset_button.on_click(lambda btn: reset_points())

fig.on_displayed(update_reg_line)
HBox([fig, VBox(check_boxes + [reset_button])])

Acquisition Functions (trading off exploration and exploitation)

1. $\large \mu(x) + \epsilon \sigma(x) $ (Exploitation plus some Exploration)
2. $\large PI(x) = \Phi(\frac{\mu(x) - \mu^+ - \epsilon}{\sigma(x)}) $ (Probabilty of improvement)
3. $\large EI(x) = (\mu(x) - \mu^+ - \epsilon)\Phi(z) + \sigma(x)\phi(z) $ &nbsp; ($\sigma(x) \ge 0$)
   <br>where $ \large z = \frac{(\mu(x) - \mu^+ - \epsilon)}{\sigma(x)} $

TODO: Implement acquisition functions and bayesian optimization algorithm

In [ ]:
# from scipy.stats import norm

# def PI(mu, sigma, eps=1):
#     f_max = np.max(mu)
#     return norm.cdf((mu - f_max - eps) / sigma)

In [ ]:
# ac_fig = plt.figure(title='Acquisition Function', 
#                     layout=Layout(width='1000px', height='300px'),
#                     fig_margin=fig_margin)

# eps = .1
# acquisition_func2 = PI(mu_post, sig_post, eps=eps)

# ac_func_line = plt.plot(X_test, [acquisition_func2],
#                         stroke_width=3)
# VBox([fig, ac_fig])